## 6-2 예제: 와인 분류하기2

지난 Chap5 5.5 와인분류 모형에서 신경망을 이용해 와인을 분류하는 모형을 만들었다.<br/>
이번에는 중간층의 수를 늘려 다층퍼셉트론을 만들어 분류 모형을 만들어보자!

* 학습 데이터를 준비하고 전처리 하는 과정은 지난 와인 분류기 노트북과 동일하다.<br/>
다시한번 내용을 살펴보면서 모델을 구현해보자

# #1.학습 데이터 준비

* Import Library - 라이브러리 불러오기
* Load Data      - 데이터 불러오기
* EDA            - 탐색적 데이터 분석

## Import Library

In [1]:
# Import Python Library

import torch
from torch.autograd import Variable 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Import Scikit-learn Library
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Import Pandas Library
import pandas as pd

* Variable(data): 텐서를 래핑하고, 계산 과정을 기록하는 역할을 한다

## Load Data

In [2]:
wine = load_wine()
# wine

In [3]:
df = pd.DataFrame(wine.data, columns=wine.feature_names, )
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


## EDA(Exploratory Data Analysis)

In [5]:
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [6]:
wine.data

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [7]:
wine.data.shape

(178, 13)

In [8]:
wine.feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [9]:
wine.feature_names.shape

AttributeError: 'list' object has no attribute 'shape'

In [10]:
len(wine.feature_names)

13

In [18]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [20]:
import numpy as np

np.unique(wine.target)

array([0, 1, 2])

In [15]:
set(wine.target)

{0, 1, 2}

Q. target 값이 0, 1, 2 3개의 값으로 이루어져 있다.이 때 target값이 2가되는 시작 인덱스의 값은 얼마인가?

# #2. Data Preprocessing

* NumPy to DataFrame
* Data preprocessing

### Numpy to DataFrame

In [16]:
df = pd.DataFrame(data = wine.data, columns=wine.feature_names)
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


178행 13열의 데이터프레임 생성

## Data Preprocessing

* 설명변수 목적변수 분할
* Train_Test_Split

### 설명변수, 목적변수 분할

In [17]:
# 설명변수와 목적변수를 변수에 대입

wine_data = wine.data[0:130]  # wine data에서 130행까지의 데이터만 사용한다.
wine_target = wine.target[0:130]

In [21]:
import numpy as np
np.unique(wine_target)

array([0, 1])

### Train_Test_Split

In [22]:
# Train_Test_Split 함수 임포트하기
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)
print('train_X.shape: ', train_X.shape)
print('train_Y.shape: ', train_Y.shape)
print('test_X.shape: ', test_X.shape)
print('test_Y.shape: ', test_Y.shape)

train_X.shape:  (104, 13)
train_Y.shape:  (104,)
test_X.shape:  (26, 13)
test_Y.shape:  (26,)


In [23]:
print(type(train_Y))

<class 'numpy.ndarray'>


# #3. Make Tensor(텐서 생성)

**Reivew** <br/>
완성된 train, test 데이터를 PyTorch의 tensor 데이터로 변환한다.<br/>
numpy.ndarray 데이터를 torch.tensor형으로 변환해 파이토치가 다룰 수 있는 형태로 정리해준다.

* 훈련 데이터와 테스트 데이터 텐서 변환
* 설명변수와 목적변수 병합
* Mini-Batch

### 훈련 데이터와 테스트 데이터 텐서 변환

In [24]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터의 형상을 확인해보자
print('train_X.shape: ', train_X.shape)
print('train_Y.shape: ', train_Y.shape)
print('test_X.shape: ', test_X.shape)
print('test_Y.shape: ', test_Y.shape)

train_X.shape:  torch.Size([104, 13])
train_Y.shape:  torch.Size([104])
test_X.shape:  torch.Size([26, 13])
test_Y.shape:  torch.Size([26])


### 설명변수와 목적변수 병합

설명변수와 목적변수의 텐서를 병합해준다. 이 때 훈련데이터만 병합한다. 왜 그럴까?<br/>
테스트 데이터는 트레이닝 과정에 사용되지 않기 때문이다. 즉, 잘 훈련된 모델의 성능을 평가할 때만 사용하기 때문이다.

In [27]:
# 설명변수와 목적변수의 텐서를 병합
# #1. 라이브러리 임포트 과정에서 불러왔던 TensorDataset()함수를 사용한다.

train = TensorDataset(train_X, train_Y)  # from torch.unils.data import DataLoader, TensorDataset
# print('train.shape: ', train.shape)

In [28]:
train[0]

(tensor([1.1820e+01, 1.4700e+00, 1.9900e+00, 2.0800e+01, 8.6000e+01, 1.9800e+00,
         1.6000e+00, 3.0000e-01, 1.5300e+00, 1.9500e+00, 9.5000e-01, 3.3300e+00,
         4.9500e+02]), tensor(1))

### Mini-Batch

batch size가 16인 미니배치로 분할을 하자<br/>
from torch.unils.data import DataLoader에서 불러온 DataLoader()함수를 사용한다.

In [29]:
# 미니배치로 분할
train_loader = DataLoader(train, batch_size=16, shuffle=True)

In [30]:
train_loader

# #4. 신경망 구성
학습에 사용할 다층퍼셉트론을 구성하는 단계이다.<br/>
이번에는 중간층이 5층인 신경망을 구성한다.<br/>

* 입력층의 노드 수는 13개
* 중간층의 노드 수는 96개
* 출력층의 노드 수는 2개
<br/>


위와 같은 신경망을 구현해보자


In [31]:
# 신경망 구성
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

# #5. 모형 학습

앞서 생성한 텐서를 신경망에 입력해 모형을 학습해본다.
학습된 모형의 정확도도 측정해보도록 한다.

In [32]:
# 오차함수 객체 
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    #분할해둔 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 게산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)  # 와우... 정말 간단하네...
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산 
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 tensor(2.1583)
100 tensor(1.5621)
150 tensor(2.0515)
200 tensor(1.3868)
250 tensor(1.2280)
300 tensor(1.5892)


# #6. Accuracy test 

In [33]:
# 계산 그리프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.8076923076923077

조금 더 학습해볼까요?

* 학습 정리
    * 1. 오차함수 객체 생성
    * 2. 최적화 객체 생성
    * 3. 학습 시작
        * 분할해온 데이터를 꺼내온다 
            * 3-1) 계산 그래프 구성
            * 3-2) 경사 소기화
            * 3-3) 순전파 계산
            * 3-4) 오차 계산
            * 3-5) 역전파 계산
            * 3-6) 가중치 업데이트
            * 3-7) 누적오차 계산
         * 50회마다 누적 오차 출력

1000에폭만큼 학습해보자 loss가 얼마나 줄어들까?

In [35]:
# 1. 오차 함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 2. 최적화 객체 생성
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 3. 학습 시작
for epoch in range(1000):
    total_loss = 0
    # 분할해온 데이터를 꺼내온다.(train_loader)에서
    for train_x, train_y in train_loader:
        # 3-1) 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 3-2) 경사 초기화
        optimizer.zero_grad()
        # 3-3) 순전파 계산
        output = model(train_x)
        # 3-4) 오차 계산
        loss = criterion(output, train_y) 
        # 3-5) 역전파 계산
        loss.backward()
        # 3-6) 가중치 업데이트 
        optimizer.step()
        # 3-7) 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차를 출력한다.
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 tensor(1.2742)
100 tensor(1.1651)
150 tensor(1.1647)
200 tensor(1.8895)
250 tensor(1.2294)
300 tensor(1.7649)
350 tensor(1.0492)
400 tensor(1.0913)
450 tensor(1.3442)
500 tensor(1.1261)
550 tensor(1.0991)
600 tensor(1.1885)
650 tensor(1.2699)
700 tensor(0.9641)
750 tensor(1.0615)
800 tensor(1.2582)
850 tensor(1.2832)
900 tensor(1.3237)
950 tensor(0.9586)
1000 tensor(1.0653)


다시 한번 Accuracy를 찍어보자<br/>
epoch수를 1000으로 늘렸을 때 정확도는 얼마나 향상될까?

In [36]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)
# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9230769230769231

epoch수를 700 늘렸더니 정확도가 무려 12% 향상되었다.

### 이번에는 Learning rate를 줄여보자
**lr를 0.001로 줄여보자**

In [37]:
# 1. 오차 함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 2. 최적화 객체 생성
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 3. 학습 시작
for epoch in range(1000):
    total_loss = 0
    # 분할해온 데이터를 꺼내온다.(train_loader)에서
    for train_x, train_y in train_loader:
        # 3-1) 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 3-2) 경사 초기화
        optimizer.zero_grad()
        # 3-3) 순전파 계산
        output = model(train_x)
        # 3-4) 오차 계산
        loss = criterion(output, train_y) 
        # 3-5) 역전파 계산
        loss.backward()
        # 3-6) 가중치 업데이트 
        optimizer.step()
        # 3-7) 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차를 출력한다.
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 tensor(0.9182)
100 tensor(0.9829)
150 tensor(0.8226)
200 tensor(0.8716)
250 tensor(0.8822)
300 tensor(0.7771)
350 tensor(0.7578)
400 tensor(0.8039)
450 tensor(0.7381)
500 tensor(0.8006)
550 tensor(0.9186)
600 tensor(0.7777)
650 tensor(0.6735)
700 tensor(0.7891)
750 tensor(0.7694)
800 tensor(0.7160)
850 tensor(0.7281)
900 tensor(0.6280)
950 tensor(0.6527)
1000 tensor(0.6457)


loss가 1.6254에서 0.6346으로 줄었다...

In [40]:
# 계산그래프 구성
test_x, test_y = Variable(test_x), Variable(test_y)
# 출력이 0또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]
# 모형의 정확도 측성
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9615384615384616

92%에서 96%로 더 상승

이상태로 epoch수를 늘려보자

In [43]:
# 1. 오차 함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 2. 최적화 객체 생성
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 3. 학습 시작
for epoch in range(2000):
    total_loss = 0
    # 분할해온 데이터를 꺼내온다.(train_loader)에서
    for train_x, train_y in train_loader:
        # 3-1) 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 3-2) 경사 초기화
        optimizer.zero_grad()
        # 3-3) 순전파 계산
        output = model(train_x)
        # 3-4) 오차 계산
        loss = criterion(output, train_y) 
        # 3-5) 역전파 계산
        loss.backward()
        # 3-6) 가중치 업데이트 
        optimizer.step()
        # 3-7) 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차를 출력한다.
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


100 tensor(1.4901e-08)
200 tensor(1.4901e-08)
300 tensor(1.4901e-08)
400 tensor(1.4901e-08)
500 tensor(1.4901e-08)
600 tensor(1.4901e-08)
700 tensor(1.4901e-08)
800 tensor(1.4901e-08)
900 tensor(1.4901e-08)
1000 tensor(1.4901e-08)
1100 tensor(2.2352e-08)
1200 tensor(2.2352e-08)
1300 tensor(1.4901e-08)
1400 tensor(1.4901e-08)
1500 tensor(2.2352e-08)
1600 tensor(1.4901e-08)
1700 tensor(2.2352e-08)
1800 tensor(1.4901e-08)
1900 tensor(1.4901e-08)
2000 tensor(1.4901e-08)


In [44]:
# 계산 그리프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9230769230769231

optimizer는 Adam이 좋다던데...
### Adam로 최적해 찾기

In [41]:
# 1. 오차 함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 2. 최적화 객체 생성
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. 학습 시작
for epoch in range(2000):
    total_loss = 0
    # 분할해온 데이터를 꺼내온다.(train_loader)에서
    for train_x, train_y in train_loader:
        # 3-1) 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 3-2) 경사 초기화
        optimizer.zero_grad()
        # 3-3) 순전파 계산
        output = model(train_x)
        # 3-4) 오차 계산
        loss = criterion(output, train_y) 
        # 3-5) 역전파 계산
        loss.backward()
        # 3-6) 가중치 업데이트 
        optimizer.step()
        # 3-7) 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차를 출력한다.
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


100 tensor(0.7274)
200 tensor(0.3738)
300 tensor(0.5605)
400 tensor(0.5831)
500 tensor(0.1633)
600 tensor(0.0017)
700 tensor(2.6754e-05)
800 tensor(6.5639e-06)
900 tensor(2.3693e-06)
1000 tensor(1.1027e-06)
1100 tensor(6.3330e-07)
1200 tensor(4.4703e-07)
1300 tensor(2.2352e-07)
1400 tensor(1.4901e-07)
1500 tensor(1.1176e-07)
1600 tensor(5.9605e-08)
1700 tensor(3.7253e-08)
1800 tensor(2.2352e-08)
1900 tensor(2.2352e-08)
2000 tensor(1.4901e-08)


역시...빠르고... 최적화도 급이 다르구나...<br/>
20.02.17.mon am 12:37

# 6. Accuracy Test
test data로 accuracy를 계산해보자

In [42]:
# 계산 그리프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9230769230769231

Adam Optimimzer를 사용해서 Accuracy를 테스트해본 결과. 의외로 Test 정확도는 낮게 나왔다.<br/>
왜때문일까?